In [1]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
from langchain_setup import ChatOpenAI, tracing_v2_enabled_if_api_key_set

# Summarization
如在 4. Data Utilization 所說，如何處理多個 documents 有數種架構，以下我們示範如何用 map_reduce 架構對多個 documents 寫出一個摘要

In [2]:
loader = NotionDirectoryLoader('../data/notion')
documents = loader.load()
splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=200)
documents = splitter.split_documents(documents=documents)

prompt = PromptTemplate(
    input_variables=["text"],
    template="請為下面的文章段落寫一句精準的摘要:\n\n\n「{text}」\n\n\n精準摘要:",
) # 原本預設的 prompt 的中文翻譯而已

chain = load_summarize_chain(
    llm=ChatOpenAI(), 
    chain_type="map_reduce", 
    verbose=False,
    map_prompt=prompt, # 為每個 document 寫一個摘要
    combine_prompt=prompt, # 對所有摘要寫一個摘要
)

這個 Chain 裡面會有兩個 chain。一個是 llm_chain，負責對每個文件產生摘要。第二個是 combine_document_chain, 會把所有摘要在摘要一次，而執行者是它底下的另一個 llm_chain。

In [3]:
print(type(chain))
print("======為每個文件寫摘要的 Prompt:======")
print(chain.llm_chain.prompt.template)
print("======對所有摘要寫摘要的 Prompt:======")
print(chain.combine_document_chain.llm_chain.prompt.template)

<class 'langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain'>
======為每個文件寫摘要的 Prompt:======
請為下面的文章段落寫一句精準的摘要:


「{text}」


精準摘要:
======對所有摘要寫摘要的 Prompt:======
請為下面的文章段落寫一句精準的摘要:


「{text}」


精準摘要:


In [4]:
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    output = chain.run(documents)
    print(output)

本文介紹了日本最具代表性的軟體開發手法-隕石落下型開發，強調了其快速但容易出現問題的特點。文章提到了敏捷式開發的循環以及隕石落下式開發法，並指出計畫是由天界的一己之願來訂定，回饋無法傳達到神的耳裡，只能透過祈禱上達天聽。文章還提到了不同形式的神明以及可能發生的諸神黃昏和聖戰事件，並指出人們只能祈禱能儘早適應這些情況。
[LangSmith URL]: https://smith.langchain.com/o/34ec837d-8405-462d-b949-fdfaebda792b/projects/p/fdcbda35-4d3a-418b-ab49-7e3205e630a6/r/6124d0d5-ae80-4733-b16e-e6c6fdb5fca6?poll=true
